In [1]:
## 安装langchain和openai
%pip install langchain==0.0.270 openai

<h3>Model I/O</h3>
<h5>1.Prompts</h5>
<h5>2.Language models</h5>
<h5>3.Output parsers</h5>


In [38]:
## load_dotenv 加载环境变量 即当前文件夹下的.env
## 可以添加配置OPENAI_API_KEY="sk-xxx"
import os
os.environ.clear()
from dotenv import load_dotenv
## 加载环境变量
load_dotenv()

True

<h6>1.1 PromptTemplate</h6>
<h6>1.2 ChatPromptTemplate</h6>

In [20]:
# PromptTemplate 

from langchain import PromptTemplate
prompt_template = PromptTemplate.from_template("please tell me what is {topic}?")
fm= prompt_template.format(topic="ModelScope")
print(fm)

## 显示的指定input_variables
prompt_tp2=PromptTemplate(template="how to use {tool} to work with {app}?", input_variables=["tool","app"])
prompt_tp2.format(tool="langchain", app="Bilibili")


please tell me what is ModelScope?


'how to use langchain to work with Bilibili?'

In [44]:
#Chat prompt template
from langchain.prompts import ChatPromptTemplate
tp =ChatPromptTemplate.from_messages([
    ("system","""you are an IT programming expert, when i ask your question you should give the accurate answer.
    Don't make it up.if you don't know, just say i have no idea
    """),
    ("human","{question}")
])
mes= tp.format_messages(
question="how to use langchain to work with Bilibili"
)

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0,openai_api_key=os.environ.get('OPENAI_API_KEY'), model_name="gpt-3.5-turbo",verbose=True)
# llm(mes)

#SystemMessage HumanMessage
from langchain.prompts.chat import SystemMessage, HumanMessage
tp2 =ChatPromptTemplate.from_messages([
    SystemMessage(content="""you are an IT programming expert, when i ask your question you should give the accurate answer.
    Don't make it up.if you don't know, just say i have no idea
    """),
    HumanMessage(content="{question}")
])
mes2=tp2.format_messages(question="""i want to use python3+langchain to write the automation tool that 
                    can connect with Bitbucket,Jira,Confluence and postman, can give me a solution
                    """)
llm(mes2)

AIMessage(content='What is the difference between front-end and back-end development?', additional_kwargs={}, example=False)

<h6>1.3 CustomPromptyTemplate</h6>

In [45]:
#Define custom prompt template
#https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/custom_prompt_template

import inspect
def get_source_code(function_name):
    #Get the source code of the function
    return inspect.getsource(function_name)


from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

PROMPT="""\
Given the function name and source code, generate an Chinese language explanation of the function.
Function Name: {function_name}
Source Code:
{source_code}
Explanation:
"""

class FunctionExplainerPromptTemplate(StringPromptTemplate,BaseModel):
    """
    A custom prompt template that takes in the function name as input, 
    and formats the prompt template to provide the source code of the function.
    """

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """
        Validate that the input variables are correct.
        """
        if len(v) !=1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        #Get the source code of the function
        source_code=get_source_code(kwargs["function_name"])

        #Generate the prompt to be sent to the language model
        prompt=PROMPT.format(function_name=kwargs["function_name"].__name__
                             ,source_code=source_code)
        return prompt

    def _prompt_type(self) -> str:
        return "function_explainer"

In [46]:
#use the custom prompt template
f_e=FunctionExplainerPromptTemplate(input_variables=["function_name"])

prompt=f_e.format(function_name=get_source_code)
print(prompt)

Given the function name and source code, generate an Chinese language explanation of the function.
Function Name: get_source_code
Source Code:
def get_source_code(function_name):
    #Get the source code of the function
    return inspect.getsource(function_name)

Explanation:



<h6>1.4 FewShotPromptTemplate</h6>
<content>Few-shot prompt 是一种在大型语言模型（LLM）中使用的技术，它可以通过提供少量的示例来帮助模型理解任务并生成相应的内容。 Few-shot prompt 通常用于那些需要模型快速学习新任务或适应新领域的场景，例如文本分类、摘要、翻译、问答等。
在这些场景中，Few-shot prompt 可以通过提供少量的输入-输出示例来帮助模型理解任务，并在不需要大量训练数据或重新训练模型的情况下快速生成相应的内容。这种方法在处理小数据集或需要快速适应新任务的场景中非常有用。</content>

In [53]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples=[
    {
     "question":"what's our team current Sprint JIRA task?",
     "answer":{"tag":"JIRA","title":"team current Sprint JIRA list",
               "path":"/v1/jira/team1","method":"GET","requstBody":{"param":1}} 
    },
    {
     "question":"what's our team last Sprint JIRA task?",
     "answer":{"tag":"JIRA","title":"team last Sprint JIRA list",
               "path":"/v1/jira/team1","method":"GET","requstBody":{"param":2}} 
    },
    {
     "question":"what's the William's JIRA task in current Sprint?",
     "answer":{"tag":"JIRA","title":"team member Sprint JIRA list",
               "path":"/v1/jira/team1","method":"GET","requstBody":{"param":3}} 
    },
    {
     "question":"what's the TODO List JIRA task in current Sprint?",
     "answer":{"tag":"JIRA","title":"team current Sprint TODO JIRA list",
               "path":"/v1/jira/team1","method":"GET","requstBody":{"param":4}} 
    },
]

In [54]:
e_p=PromptTemplate(input_variables=["question","answer"],template="Question: {question}\n{answer}")
print(e_p.format(**examples[3]))

Question: what's the TODO List JIRA task in current Sprint?
{'tag': 'JIRA', 'title': 'team current Sprint TODO JIRA list', 'path': '/v1/jira/team1', 'method': 'GET', 'requstBody': {'param': 4}}


In [59]:
# Feed examples and formatter to FewShotPromptTemplate
prompt=FewShotPromptTemplate(
    examples=examples,
    # example_prompt=e_p,
    # suffix="Question: {input}",
    # input_variables=["input"]
)
print(prompt)
#print(prompt.format(input="what's the Cancelled List JIRA task in current Sprint?"))

KeyError: 'input_variables'